In [1]:
# Importing libraries and packages
import numpy as np
import mediapipe as mp
import cv2


In [2]:
# Connecting keypoints visuals
mp_drawing = mp.solutions.drawing_utils

# Keypoint detection
mp_pose = mp.solutions.pose

# Define constants
POSE_LANDMARKS = mp_pose.PoseLandmark
NUM_KEYPOINTS = len(POSE_LANDMARKS)


In [3]:
# Calculate angle between three points
def calc_angle(a, b, c):
    a = np.array([a.x, a.y])
    b = np.array([b.x, b.y])
    c = np.array([c.x, c.y])

    ab = np.subtract(a, b)
    bc = np.subtract(b, c)

    theta = np.arccos(np.dot(ab, bc) / (np.linalg.norm(ab) * np.linalg.norm(bc)))
    theta = 180 - 180 * theta / np.pi
    return np.round(theta, 2)

In [13]:
# Initialize variables
flag = None
count = 0

# Video capture
cap = cv2.VideoCapture(0)

# Initializing pose
pose = mp_pose.Pose(
    min_detection_confidence=0.5,
    min_tracking_confidence=0.5,
)

while cap.isOpened():
    _, frame = cap.read()

    # Recoloring the image to RGB
    image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    image.flags.writeable = False

    # Detection
    results = pose.process(image)

    # Recoloring back to BGR
    image.flags.writeable = True
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

    # Extracting landmarks
    try:
        landmarks = results.pose_landmarks.landmark

        # Example: Calculate the angle between left shoulder, left elbow, and left wrist
        left_shoulder = landmarks[POSE_LANDMARKS.LEFT_SHOULDER.value]
        left_elbow = landmarks[POSE_LANDMARKS.LEFT_ELBOW.value]
        left_wrist = landmarks[POSE_LANDMARKS.LEFT_WRIST.value]
        angle = calc_angle(left_shoulder, left_elbow, left_wrist)

        # Visualizing the angle
        cv2.putText(
            image,
            f'Angle: {angle} degrees',
            tuple(np.multiply([left_elbow.x, left_elbow.y], [640, 480]).astype(int)),
            cv2.FONT_HERSHEY_SIMPLEX,
            0.5,
            (255, 255, 255),
            2,
            cv2.LINE_AA,
        )

        # Curl counter logic
        if angle > 160:
            flag = "down"
        if angle < 30 and flag == "down":
            count += 1
            flag = "up"

        # Display the count 
        cv2.putText(
            image,
            f'COUNT: {count}',
            (50, 50),
            cv2.FONT_HERSHEY_SIMPLEX,
            0.5,
            (173, 216, 230),
            2,
            cv2.LINE_AA,
        )

    except Exception as e:
        pass

    # Render detections
    mp_drawing.draw_landmarks(
        image,
        results.pose_landmarks,
        mp_pose.POSE_CONNECTIONS,
        mp_drawing.DrawingSpec(color=(66, 245, 117), thickness=2, circle_radius=2),  # Light green line color
        mp_drawing.DrawingSpec(color=(173, 216, 230), thickness=2, circle_radius=2),  # Bluish point color
    )

    cv2.imshow('Biceps Curl Counter', image)

    if cv2.waitKey(10) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()